<a href="https://colab.research.google.com/github/k-chanez/Churn-prediction/blob/lysa_dev/projetCloud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install google-cloud-bigquery

In [14]:
import pandas as pd
from google.cloud import bigquery

In [15]:

# Set the path to your service account key file on the GCE VM
key_path = "/projet-m2-ia-churn-prediction-db53d6498522.json"

# Set the environment variable for authentication
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = key_path
from google.cloud import bigquery

# Remplacez 'project_id' par l'ID de votre projet GCP
project_id = 'projet-m2-ia-churn-prediction'

# Exécutez une requête BigQuery
query = """
SELECT *
FROM `projet-m2-ia-churn-prediction.Ecommerce_Churn.Ecommerce_Table1`"""
df = bigquery.Client(project=project_id).query(query).to_dataframe()

# Affichez le résultat
print(df)

      CustomerID  Churn  Tenure PreferredLoginDevice  CityTier  \
0          52720      0    60.0         Mobile Phone         3   
1          50929      0    50.0         Mobile Phone         1   
2          53744      0    51.0         Mobile Phone         1   
3          55535      0    61.0         Mobile Phone         3   
4          51851      0     0.0         Mobile Phone         1   
...          ...    ...     ...                  ...       ...   
5054       53361      0     1.0         Mobile Phone         1   
5055       54286      0     1.0         Mobile Phone         1   
5056       54165      0     1.0         Mobile Phone         1   
5057       53354      1     1.0         Mobile Phone         1   
5058       54824      1     1.0         Mobile Phone         1   

      WarehouseToHome PreferredPaymentMode  Gender  HourSpendOnApp  \
0                27.0           Debit Card  Female             3.0   
1                 6.0           Debit Card    Male             3.0 

Pré-traitement de la donnée




In [27]:
# Id de la dataset
dataset_id = "ecommerce_data_churn"

# Remplacer les valeurs manquantes par la moyenne de la colonne pour les colonnes numériques

for column in df.select_dtypes(include=['int64', 'float64']):
    if pd.api.types.is_numeric_dtype(column):
        df.fillna(df[column].mean(), inplace=True)

# Remplacer les valeurs manquantes pour les colonnes categorielles avec le mod
for column in df.select_dtypes(exclude=['int64', 'float64']):
    df.fillna(df[column].mode()[0], inplace=True)

# Convertir les colonnes numériques avec des valeurs non numériques en valeurs numériques
numeric_columns = ['WarehouseToHome', 'HourSpendOnApp', 'OrderAmountHikeFromlastYear', 'CouponUsed', 'OrderCount']
for column in numeric_columns:
    df[column] = pd.to_numeric(df[column], errors='coerce')

# Traiter les valeurs manquantes pour les colonnes numériques
for column in numeric_columns:
    df[column].fillna(df[column].mean(), inplace=True)

# Traiter les valeurs incorrectes pour les colonnes catégorielles
categorical_columns = ['PreferredLoginDevice', 'PreferredPaymentMode', 'Gender', 'PreferedOrderCat', 'MaritalStatus']
for column in categorical_columns:
    df[column] = df[column].replace('Mobile Phone', df[column].mode()[0])

print(df)

      CustomerID  Churn  Tenure PreferredLoginDevice  CityTier  \
0          52720      0    60.0         Mobile Phone         3   
1          50929      0    50.0         Mobile Phone         1   
2          53744      0    51.0         Mobile Phone         1   
3          55535      0    61.0         Mobile Phone         3   
4          51851      0     0.0         Mobile Phone         1   
...          ...    ...     ...                  ...       ...   
5054       53361      0     1.0         Mobile Phone         1   
5055       54286      0     1.0         Mobile Phone         1   
5056       54165      0     1.0         Mobile Phone         1   
5057       53354      1     1.0         Mobile Phone         1   
5058       54824      1     1.0         Mobile Phone         1   

      WarehouseToHome PreferredPaymentMode  Gender  HourSpendOnApp  \
0                27.0           Debit Card  Female             3.0   
1                 6.0           Debit Card    Male             3.0 

In [28]:
print(df.describe())


         CustomerID     Churn       Tenure  CityTier  WarehouseToHome  \
count        5059.0    5059.0  5059.000000    5059.0      5059.000000   
mean   52847.095671  0.160704    10.098438  1.651117        15.669925   
std     1621.175136  0.367294     8.570746  0.914842         8.376981   
min         50001.0       0.0     0.000000       1.0         5.000000   
25%         51441.5       0.0     2.000000       1.0         9.000000   
50%         52876.0       0.0     9.000000       1.0        14.000000   
75%         54253.5       0.0    16.000000       3.0        20.000000   
max         55630.0       1.0    61.000000       3.0       127.000000   

       HourSpendOnApp  NumberOfDeviceRegistered  SatisfactionScore  \
count     5059.000000                    5059.0             5059.0   
mean         2.942132                  3.684523           3.061277   
std          0.705835                  1.031387           1.384473   
min          0.000000                       1.0               

In [29]:
print(df.isnull().sum())


CustomerID                     0
Churn                          0
Tenure                         0
PreferredLoginDevice           0
CityTier                       0
WarehouseToHome                0
PreferredPaymentMode           0
Gender                         0
HourSpendOnApp                 0
NumberOfDeviceRegistered       0
PreferedOrderCat               0
SatisfactionScore              0
MaritalStatus                  0
NumberOfAddress                0
Complain                       0
OrderAmountHikeFromlastYear    0
CouponUsed                     0
OrderCount                     0
DaySinceLastOrder              0
CashbackAmount                 0
dtype: int64


In [30]:
for column in df.select_dtypes(exclude=['int64', 'float64']):
    print(f"Valeurs uniques pour {column}: {df[column].unique()}")


Valeurs uniques pour PreferredLoginDevice: ['Mobile Phone' 'Computer']
Valeurs uniques pour PreferredPaymentMode: ['Debit Card' 'Credit Card' 'Cash on Delivery' 'UPI' 'E wallet']
Valeurs uniques pour Gender: ['Female' 'Male']
Valeurs uniques pour PreferedOrderCat: ['Grocery' 'Laptop & Accessory' 'Mobile' 'Fashion' 'Others']
Valeurs uniques pour MaritalStatus: ['Married' 'Divorced' 'Single']


In [31]:
print(df.head())


   CustomerID  Churn  Tenure PreferredLoginDevice  CityTier  WarehouseToHome  \
0       52720      0    60.0         Mobile Phone         3             27.0   
1       50929      0    50.0         Mobile Phone         1              6.0   
2       53744      0    51.0         Mobile Phone         1              7.0   
3       55535      0    61.0         Mobile Phone         3             28.0   
4       51851      0     0.0         Mobile Phone         1             12.0   

  PreferredPaymentMode  Gender  HourSpendOnApp  NumberOfDeviceRegistered  \
0           Debit Card  Female             3.0                         3   
1           Debit Card    Male             3.0                         1   
2           Debit Card    Male             4.0                         2   
3           Debit Card  Female             4.0                         4   
4           Debit Card  Female             3.0                         3   

     PreferedOrderCat  SatisfactionScore MaritalStatus  Number

In [42]:

# Spécifiez l'ID du projet Google Cloud et l'ID du dataset et de la table
project_id = 'votre-projet-id'
dataset_id = 'votre-dataset-id'
table_id = 'votre-table-id'

# Créez un client BigQuery
client = bigquery.Client(project=project_id)

# Liste des colonnes catégorielles (remplacez par vos colonnes)
categorical_columns = ['PreferredLoginDevice', 'PreferredPaymentMode', 'Gender', 'PreferedOrderCat', 'MaritalStatus']

# Générez la requête SQL pour obtenir les statistiques descriptives pour les colonnes catégorielles
query_categorical = """
SELECT
  {columns},
  COUNT(*) AS count,
  COUNT(DISTINCT {first_column}) AS unique_count
FROM
  `{project_id}.{dataset_id}.{table_id}`
GROUP BY
  {columns}
""".format(
    columns=", ".join(categorical_columns),
    first_column=categorical_columns[0],
    project_id=project_id,
    dataset_id=dataset_id,
    table_id=table_id
)

# Exécutez la requête
query_job_categorical = client.query(query_categorical)
result_categorical = query_job_categorical.to_dataframe()
# Assurez-vous que les colonnes catégorielles correspondent exactement aux noms dans votre table
for column in categorical_columns:
    if column not in result_categorical.columns:
        print(f"La colonne '{column}' n'a pas été trouvée dans le résultat de la requête.")


# Affichez les statistiques descriptives
print(result_categorical)


BadRequest: ignored